# Exploratory Data Analysis Final Project (Spotify Tracks)

**Dataset:** Spotify Tracks Dataset (Kaggle)  
**Goal:** Clean the dataset, engineer a few helpful features, explore patterns with EDA visualizations, and test at least one hypothesis with a statistical significance test.

> Note: Spotify audio features such as *danceability, energy, acousticness, valence,* etc. are defined on a **0–1** scale in Spotify’s Audio Features documentation.  
Reference: https://developer.spotify.com/documentation/web-api/reference/get-audio-features


In [ ]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

# Load data (make sure dataset.csv is in the same folder as this notebook)
df = pd.read_csv("dataset.csv")
df.head()

## 1) Dataset summary

In this section we capture:
- Dataset size (rows × columns)
- Variables (column names + types)
- Candidate target variables for analysis / ML

**Possible target variables (depending on the goal):**
- `popularity` (regression target)
- `track_genre` (classification target)
- `explicit` (binary classification target)


In [ ]:
# Size and basic structure
df.shape

In [ ]:
# Column names and data types
df.info()

In [ ]:
# Quick descriptive statistics for numeric columns
df.describe().T

## 2) Data exploration plan (high level)

Planned steps:
1. Check missing values and duplicates (including repeated `track_id`).
2. Remove technical columns that do not represent real features (e.g., exported index columns).
3. Create a **track-level** dataset (`df_track`) where each `track_id` appears once (helps independence for hypothesis tests).
4. Engineer a few simple features (e.g., duration in minutes).
5. EDA:
   - Distributions (popularity, duration)
   - Correlations among audio features
   - Comparisons by genre (top 10) and explicitness
6. Hypotheses + one deep significance test (Welch t-test or ANOVA/Kruskal depending on the hypothesis).


## 3) Data quality checks (missing values & duplicates)

We check:
- Missing values per column
- Exact duplicate rows
- Repeated `track_id` (the same track can appear multiple times in a genre-labeled dataset)


In [ ]:
# Missing values
df.isna().sum().sort_values(ascending=False).head(10)

In [ ]:
# Exact duplicate rows (all columns identical)
df.duplicated().sum()

In [ ]:
# How often does track_id repeat?
df["track_id"].duplicated().sum(), df["track_id"].nunique(), len(df)

In [ ]:
# Most repeated track_ids (to understand the level of repetition)
df["track_id"].value_counts().head(10)

## 4) Data cleaning & creating a track-level dataset

Cleaning decisions (typical for this dataset):
- Drop technical/index columns (commonly `Unnamed: 0` in Kaggle exports).
- Drop exact duplicate rows.
- Keep a **track-level** table `df_track` with one row per `track_id`:
  - We keep the row with the **highest popularity** when a track appears multiple times.
  - This helps avoid counting the same track multiple times in EDA and hypothesis tests.


In [ ]:
# 1) Drop technical columns if present
df_clean = df.drop(columns=["Unnamed: 0", "index"], errors="ignore").copy()

# 2) Drop exact duplicate rows
df_clean = df_clean.drop_duplicates()

# 3) Create track-level dataset (one row per track_id)
df_track = (df_clean
            .sort_values("popularity", ascending=False)
            .drop_duplicates(subset="track_id")
            .copy())

df_clean.shape, df_track.shape

In [ ]:
# Verify no missing values remain in df_track (optional)
df_track.isna().sum().sort_values(ascending=False).head(10)

## 5) Feature engineering

Simple features to improve interpretability and analysis:
- `duration_min`: convert milliseconds to minutes
- `mode_name`: map `mode` (0/1) to Minor/Major
- convert `explicit` boolean to 0/1 (useful for plotting and hypothesis tests)
- `pop_bin`: popularity quartiles for group comparisons


In [ ]:
df_track["duration_min"] = df_track["duration_ms"] / 60000
df_track["mode_name"] = df_track["mode"].map({1: "Major", 0: "Minor"})
df_track["explicit_int"] = df_track["explicit"].astype(int)
df_track["pop_bin"] = pd.qcut(df_track["popularity"], q=4, labels=["low", "mid", "high", "top"])

df_track[["duration_ms", "duration_min", "mode", "mode_name", "explicit", "explicit_int", "popularity", "pop_bin"]].head()

## 6) EDA visualizations

Recommended EDA screenshots for the PDF report:
- Popularity distribution
- Duration distribution
- Correlation heatmap of audio features
- Popularity by top 10 genres
- Popularity by explicit vs non-explicit


In [ ]:
# Popularity distribution
plt.figure(figsize=(8,4))
sns.histplot(data=df_track, x="popularity", bins=30)
plt.title("Popularity distribution")
plt.tight_layout()
plt.show()

In [ ]:
# Duration distribution (minutes) — note possible long-track outliers
plt.figure(figsize=(8,4))
sns.histplot(data=df_track, x="duration_min", bins=30)
plt.title("Track duration (minutes)")
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap (numeric audio features)
num_cols = ["danceability","energy","speechiness","acousticness",
            "instrumentalness","liveness","valence","tempo","loudness","popularity"]

corr = df_track[num_cols].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, vmin=-1, vmax=1)
plt.title("Correlation heatmap (selected numeric features)")
plt.tight_layout()
plt.show()

In [ ]:
# Genre comparisons (top 10 by count)
top_genres = df_track["track_genre"].value_counts().head(10).index
df_top = df_track[df_track["track_genre"].isin(top_genres)].copy()

plt.figure(figsize=(12,5))
sns.boxplot(data=df_top, x="track_genre", y="popularity")
plt.xticks(rotation=45, ha="right")
plt.title("Popularity by top 10 genres (boxplot)")
plt.tight_layout()
plt.show()

In [ ]:
# Explicit vs non-explicit popularity
plt.figure(figsize=(6,4))
sns.boxplot(data=df_track, x="explicit_int", y="popularity")
plt.xticks([0,1], ["Non-explicit (0)", "Explicit (1)"])
plt.title("Popularity: explicit vs non-explicit")
plt.tight_layout()
plt.show()

## 7) Hypotheses (at least 3)

Example hypotheses for this dataset:

**H1 (Explicitness):**  
- H0: Mean popularity is the same for explicit and non-explicit tracks.  
- H1: Mean popularity differs between explicit and non-explicit tracks.

**H2 (Genres):**  
- H0: Mean popularity is the same across the top 10 genres.  
- H1: At least one genre has a different mean popularity.

**H3 (Audio features vs popularity):**  
- H0: Danceability has no association with popularity.  
- H1: Danceability is positively associated with popularity.

In the next section we run a significance test for one strong hypothesis (H1).


## 8) Hypothesis testing & significance analysis (Welch t-test)

We test H1 using **Welch’s t-test**, which does **not** assume equal variances between groups (a safer choice when group variances or sizes may differ).
We report:
- p-value (statistical significance)
- group means/medians (interpretability)
- effect size (Cohen’s d) to quantify practical magnitude


In [ ]:
from scipy import stats

# Split groups
a = df_track[df_track["explicit_int"]==1]["popularity"]
b = df_track[df_track["explicit_int"]==0]["popularity"]

# Welch t-test
t_stat, p_value = stats.ttest_ind(a, b, equal_var=False)
t_stat, p_value

In [ ]:
# Effect size: Cohen's d (pooled standard deviation version)
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    sx, sy = x.std(ddof=1), y.std(ddof=1)
    sp = np.sqrt(((nx-1)*sx**2 + (ny-1)*sy**2) / (nx+ny-2))
    return (x.mean() - y.mean()) / sp

results = {
    "Welch t-test p-value": p_value,
    "Cohen's d": cohens_d(a, b),
    "Mean (Explicit=1)": a.mean(),
    "Mean (Explicit=0)": b.mean(),
    "Median (Explicit=1)": a.median(),
    "Median (Explicit=0)": b.median(),
    "n (Explicit=1)": len(a),
    "n (Explicit=0)": len(b),
}
results

## 9) Key findings & insights (write after you review the plots)

Use 5–8 stakeholder-friendly bullet points, for example:
- Overall popularity is concentrated around the low-to-mid range (see histogram).
- Some audio features show stronger relationships with popularity than others (see correlation heatmap).
- Popularity differs across top genres (boxplot shows variability and outliers).
- Explicit tracks have a statistically significant popularity difference; effect size should be interpreted (small/medium/large).

(Replace these placeholders with your own final bullets after reviewing the outputs.)


## 10) Conclusion & next steps

Conclude with:
- 3–5 key takeaways from EDA + hypothesis test
- Next steps for ML readiness (examples):
  - Create train/test splits grouped by `track_id` (to prevent leakage if duplicates exist in other views).
  - Try a simple regression model predicting `popularity`.
  - Try a genre classifier using audio features (after encoding `track_genre`).
  - Consider handling very long-duration outliers if needed.
